In [ ]:
!pip install numpy opencv-python SimpleITK nibabel pillow scikit-learn tensorflow matplotlib deap


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 13.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Part1

In [ ]:
import os
import numpy as np
import cv2
import SimpleITK as sitk
from google.colab import drive

# Mount Google Drive
print("Mounting Google Drive...")
drive.mount('/content/drive')

# Define 2D data dimensions
width, height = (224, 224)

# Data paths
data_folder = "/content/drive/MyDrive/AD_Dataset_2"
preprocessed_folder = "/content/drive/MyDrive/AD_Dataset_Preprocessed"

# Mapping class names to labels
class_name_to_label = {'VeryMildDemented': 0, 'MildDemented': 1, 'ModerateDemented': 2, 'NonDemented': 3}

# Ensure the preprocessed folder exists
os.makedirs(preprocessed_folder, exist_ok=True)

# Function to convert JPEG image to NIfTI format
def convert_jpeg_to_nifti(jpeg_path, output_nifti_path):
    print(f"Converting {jpeg_path} to NIfTI format...")
    image = cv2.imread(jpeg_path, cv2.IMREAD_GRAYSCALE)  # Read JPEG image as grayscale
    sitk_image = sitk.GetImageFromArray(image)  # Convert numpy array to SimpleITK image
    sitk.WriteImage(sitk_image, output_nifti_path)  # Save as NIfTI file

# Preprocessing functions
def skull_strip(image):
    print("Applying skull stripping...")
    brain_mask = image > 0  # Simple threshold-based mask
    brain_image = image * brain_mask
    return brain_image

def intensity_normalization(image):
    print("Normalizing intensity...")
    mean = np.mean(image)
    std = np.std(image)
    return (image - mean) / std

def bias_field_correction(image_path):
    print(f"Applying bias field correction to {image_path}...")
    image = sitk.ReadImage(image_path)
    image = sitk.Cast(image, sitk.sitkFloat32)  # Cast to supported pixel type
    corrected_image = sitk.N4BiasFieldCorrection(image)
    return sitk.GetArrayFromImage(corrected_image)  # Return 2D array directly

def resize_image(image, target_size):
    print(f"Resizing image to {target_size}...")
    return cv2.resize(image, target_size)

def normalize_image(image):
    print("Normalizing image to range [0, 1]...")
    return image / 255.0

def equalize_histogram(image):
    print("Equalizing histogram...")
    if len(image.shape) == 3 and image.shape[2] == 3:  # Color image
        img_yuv = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
        img_yuv[:, :, 0] = cv2.equalizeHist(img_yuv[:, :, 0])
        return cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)
    else:  # Grayscale image
        if image.dtype != np.uint8:
            image = (image * 255).astype(np.uint8)  # Scale and convert to uint8
        return cv2.equalizeHist(image)

# Load and preprocess data
def preprocess_and_save_data(data_folder, preprocessed_folder):
    for class_folder in os.listdir(data_folder):
        if os.path.isdir(os.path.join(data_folder, class_folder)):
            class_label = class_name_to_label.get(class_folder, -1)
            if class_label != -1:
                class_output_folder = os.path.join(preprocessed_folder, class_folder)
                os.makedirs(class_output_folder, exist_ok=True)
                for file_name in os.listdir(os.path.join(data_folder, class_folder)):
                    if file_name.endswith(".jpg"):
                        jpeg_path = os.path.join(data_folder, class_folder, file_name)
                        nifti_path = os.path.join(class_output_folder, file_name.replace(".jpg", ".nii.gz"))
                        preprocessed_image_path = os.path.join(class_output_folder, file_name.replace(".jpg", ".png"))

                        # Check if the preprocessed PNG file already exists
                        if os.path.exists(preprocessed_image_path):
                            print(f"Skipping {jpeg_path} as it has already been processed.")
                            continue

                        convert_jpeg_to_nifti(jpeg_path, nifti_path)

                        image_2d = bias_field_correction(nifti_path)
                        image_2d = skull_strip(image_2d)
                        image_2d = intensity_normalization(image_2d)
                        image_2d = resize_image(image_2d, (width, height))
                        image_2d = normalize_image(image_2d)
                        image_2d = equalize_histogram(image_2d)

                        cv2.imwrite(preprocessed_image_path, (image_2d * 255).astype(np.uint8))  # Save as PNG image

print("Preprocessing and saving data...")
preprocess_and_save_data(data_folder, preprocessed_folder)
print("Data preprocessing and saving complete.")


Streaming output truncated to the last 5000 lines.
Skipping /content/drive/MyDrive/AD_Dataset_2/NonDemented/nonDem1894.jpg as it has already been processed.
Skipping /content/drive/MyDrive/AD_Dataset_2/NonDemented/nonDem2114.jpg as it has already been processed.
Skipping /content/drive/MyDrive/AD_Dataset_2/NonDemented/nonDem2183.jpg as it has already been processed.
Skipping /content/drive/MyDrive/AD_Dataset_2/NonDemented/nonDem2383.jpg as it has already been processed.
Skipping /content/drive/MyDrive/AD_Dataset_2/NonDemented/nonDem1512.jpg as it has already been processed.
Skipping /content/drive/MyDrive/AD_Dataset_2/NonDemented/nonDem2074.jpg as it has already been processed.
Skipping /content/drive/MyDrive/AD_Dataset_2/NonDemented/nonDem1037.jpg as it has already been processed.
Skipping /content/drive/MyDrive/AD_Dataset_2/NonDemented/nonDem155.jpg as it has already been processed.
Skipping /content/drive/MyDrive/AD_Dataset_2/NonDemented/nonDem1418.jpg as it has already been process

In [ ]:
Part2.2 DENSEnet #fullcode

############

In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from google.colab import drive
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import gc
from deap import base, creator, tools

# Mount Google Drive
print("Mounting Google Drive...")
drive.mount('/content/drive')

# Define 2D data dimensions
width, height = (224, 224)

# Data paths
preprocessed_folder = "/content/drive/MyDrive/AD_Dataset_Preprocessed"
output_folder = "/content/drive/MyDrive/Models"
num_epochs = 10
batch_size = 32

# Mapping class names to labels
class_name_to_label = {'VeryMildDemented': 0, 'MildDemented': 1, 'ModerateDemented': 2, 'NonDemented': 3}

# Function to load preprocessed data
def load_preprocessed_data(preprocessed_folder):
    data, labels = [], []
    for class_folder in os.listdir(preprocessed_folder):
        class_label = class_name_to_label.get(class_folder, -1)
        if class_label != -1:
            class_folder_path = os.path.join(preprocessed_folder, class_folder)
            for file_name in os.listdir(class_folder_path):
                if file_name.endswith(".png"):
                    image_path = os.path.join(class_folder_path, file_name)
                    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
                    image = cv2.resize(image, (width, height))
                    # Add channel dimension
                    image = np.expand_dims(image, axis=-1)
                    data.append(image)
                    labels.append(class_label)

    data = np.array(data)
    labels = np.array(labels)
    print(f"Data loading complete. Loaded {len(data)} images.")
    return data, labels

print("Loading preprocessed data...")
data, labels = load_preprocessed_data(preprocessed_folder)

print("Performing train-validation split...")
train_data, val_data, train_labels, val_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

print("Defining data augmentation generator...")
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

num_classes = len(np.unique(train_labels))

def create_model(learning_rate):
    print(f"Creating model with learning rate: {learning_rate}")
    input_shape = (width, height, 1)  # Input shape for grayscale images
    base_model = tf.keras.applications.DenseNet121(weights=None, include_top=False, input_shape=input_shape)
    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

print("Setting up distributed training strategy...")
strategy = tf.distribute.experimental.CentralStorageStrategy()

def preprocess_for_tf_datagen(image, label):
    """Preprocesses an image for TensorFlow data augmentation.

    Args:
        image: A tensor representing an image.
        label: A tensor representing the image label.

    Returns:
        A tuple of (image, label) where the image is preprocessed.
    """

    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    image = tf.image.resize(image, [width, height])
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    image = tf.image.random_brightness(image, 0.2)

    # Skip hue and saturation adjustments since the image is grayscale
    return image, label


with strategy.scope():
    train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels)).map(
        preprocess_for_tf_datagen
    ).batch(batch_size).prefetch(tf.data.AUTOTUNE)

    val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_labels)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

def lr_schedule(epoch):
    return 0.001 * (0.1 ** int(epoch / 10))

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

def evaluate_model(individual):
    learning_rate = individual[0]
    print(f"Evaluating model with learning rate: {learning_rate}")
    model = create_model(learning_rate)
    model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.LearningRateScheduler(lr_schedule), early_stopping], verbose=0)
    _, val_accuracy = model.evaluate(val_dataset, verbose=0)
    del model
    gc.collect()  # Manually trigger garbage collection to clear memory
    return -val_accuracy,

del data
del labels
del train_data
del val_data

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register("attr_float", np.random.uniform, 1e-5, 1e-3)  # Learning rate range
toolbox.register("individual", tools.initCycle, creator.Individual, (toolbox.attr_float,), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

population = toolbox.population(n=10)

hall_of_fame = tools.HallOfFame(1)

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("max", np.max)
stats.register("avg", np.mean)

NGEN = 1  # Number of generations
CXPB = 0.7  # Crossover probability
MUTPB = 0.2  # Mutation probability

logbook = tools.Logbook()
logbook.header = ["gen", "evals", "max", "avg"]

for gen in range(NGEN):
    print(f"Generation {gen}")
    fitnesses = map(evaluate_model, population)
    for ind, fit in zip(population, fitnesses):
        ind.fitness.values = fit

    hall_of_fame.update(population)

    offspring = list(map(toolbox.clone, population))

    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if np.random.rand() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if np.random.rand() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    fitnesses = map(evaluate_model, offspring)
    for ind, fit in zip(offspring, fitnesses):
        ind.fitness.values = fit

    population[:] = offspring

    record = stats.compile(population)
    logbook.record(gen=gen, evals=len(population), **record)
    print(logbook.stream)

best_ind = hall_of_fame[0]
best_fitness = best_ind.fitness.values[0]
print("Best Individual:")
print("Learning Rate =", best_ind[0])
print("Best Fitness =", best_fitness)

best_learning_rate = best_ind[0]
with strategy.scope():
    final_model = create_model(best_learning_rate)

print("Training final model...")
final_model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.LearningRateScheduler(lr_schedule), early_stopping])

print("Evaluating final model...")
val_loss, val_accuracy = final_model.evaluate(val_dataset, verbose=0)
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

print("Saving final model...")
final_model.save(os.path.join(output_folder, "DenseNet121_model_2d.h5"))
print("Model saved successfully.")


Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading preprocessed data...
Data loading complete. Loaded 6400 images.
Performing train-validation split...
Defining data augmentation generator...
Setting up distributed training strategy...
Generation 0
Evaluating model with learning rate: 0.0008364937115004041
Creating model with learning rate: 0.0008364937115004041
Evaluating model with learning rate: 0.0007408422894263717
Creating model with learning rate: 0.0007408422894263717
Evaluating model with learning rate: 0.0009311736989189874
Creating model with learning rate: 0.0009311736989189874
Evaluating model with learning rate: 0.0003436449398742601
Creating model with learning rate: 0.0003436449398742601
Evaluating model with learning rate: 7.487960863826404e-05
Creating model with learning rate: 7.487960863826404e-05
Evaluating model with learning rate: 0.0007573531575782453
C

Validation Loss: 1.1049
Validation Accuracy: 52.42%
Saving final model...
Model saved successfully.


############

PART 2.3 full code (RESNET)

In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from google.colab import drive
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import gc
from deap import base, creator, tools

# Mount Google Drive
print("Mounting Google Drive...")
drive.mount('/content/drive')

# Define 2D data dimensions
width, height = (224, 224)

# Data paths
preprocessed_folder = "/content/drive/MyDrive/AD_Dataset_Preprocessed"
output_folder = "/content/drive/MyDrive/Models"
num_epochs = 20
batch_size = 32

# Mapping class names to labels
class_name_to_label = {'VeryMildDemented': 0, 'MildDemented': 1, 'ModerateDemented': 2, 'NonDemented': 3}

# Function to load preprocessed data
def load_preprocessed_data(preprocessed_folder):
    data, labels = [], []
    for class_folder in os.listdir(preprocessed_folder):
        class_label = class_name_to_label.get(class_folder, -1)
        if class_label != -1:
            class_folder_path = os.path.join(preprocessed_folder, class_folder)
            for file_name in os.listdir(class_folder_path):
                if file_name.endswith(".png"):
                    image_path = os.path.join(class_folder_path, file_name)
                    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
                    image = cv2.resize(image, (width, height))
                    # Add channel dimension
                    image = np.expand_dims(image, axis=-1)
                    data.append(image)
                    labels.append(class_label)

    data = np.array(data)
    labels = np.array(labels)
    print(f"Data loading complete. Loaded {len(data)} images.")
    return data, labels

print("Loading preprocessed data...")
data, labels = load_preprocessed_data(preprocessed_folder)

print("Performing train-validation split...")
train_data, val_data, train_labels, val_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

print("Defining data augmentation generator...")
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

num_classes = len(np.unique(train_labels))

def create_model(learning_rate):
    print(f"Creating ResNet model with learning rate: {learning_rate}")
    input_shape = (width, height, 1)  # Input shape for grayscale images
    base_model = tf.keras.applications.ResNet50(weights=None, include_top=False, input_shape=input_shape)
    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

print("Setting up distributed training strategy using MirroredStrategy...")
strategy = tf.distribute.MirroredStrategy()

def preprocess_for_tf_datagen(image, label):
    """Preprocesses an image for TensorFlow data augmentation.

    Args:
        image: A tensor representing an image.
        label: A tensor representing the image label.

    Returns:
        A tuple of (image, label) where the image is preprocessed.
    """
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    image = tf.image.resize(image, [width, height])
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    image = tf.image.random_brightness(image, 0.2)
    # Skip hue and saturation adjustments since the image is grayscale
    return image, label

with strategy.scope():
    train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels)).map(
        preprocess_for_tf_datagen
    ).batch(batch_size).prefetch(tf.data.AUTOTUNE)

    val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_labels)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

def lr_schedule(epoch):
    return 0.001 * (0.1 ** int(epoch / 10))

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

def evaluate_model(individual):
    learning_rate = individual[0]
    print(f"Evaluating model with learning rate: {learning_rate}")
    model = create_model(learning_rate)
    model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.LearningRateScheduler(lr_schedule), early_stopping], verbose=0)
    _, val_accuracy = model.evaluate(val_dataset, verbose=0)
    del model
    gc.collect()  # Manually trigger garbage collection to clear memory
    return -val_accuracy,

del data
del labels
del train_data
del val_data

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register("attr_float", np.random.uniform, 1e-5, 1e-3)  # Learning rate range
toolbox.register("individual", tools.initCycle, creator.Individual, (toolbox.attr_float,), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

population = toolbox.population(n=10)

hall_of_fame = tools.HallOfFame(1)

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("max", np.max)
stats.register("avg", np.mean)

NGEN = 1  # Number of generations
CXPB = 0.7  # Crossover probability
MUTPB = 0.2  # Mutation probability

logbook = tools.Logbook()
logbook.header = ["gen", "evals", "max", "avg"]

for gen in range(NGEN):
    print(f"Generation {gen}")
    fitnesses = map(evaluate_model, population)
    for ind, fit in zip(population, fitnesses):
        ind.fitness.values = fit

    hall_of_fame.update(population)

    offspring = list(map(toolbox.clone, population))

    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if np.random.rand() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if np.random.rand() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    fitnesses = map(evaluate_model, offspring)
    for ind, fit in zip(offspring, fitnesses):
        ind.fitness.values = fit

    population[:] = offspring

    record = stats.compile(population)
    logbook.record(gen=gen, evals=len(population), **record)
    print(logbook.stream)

best_ind = hall_of_fame[0]
best_fitness = best_ind.fitness.values[0]
print("Best Individual:")
print("Learning Rate =", best_ind[0])
print("Best Fitness =", best_fitness)

best_learning_rate = best_ind[0]
with strategy.scope():
    final_model = create_model(best_learning_rate)

print("Training final model...")
final_model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.LearningRateScheduler(lr_schedule), early_stopping])

print("Evaluating final model...")
val_loss, val_accuracy = final_model.evaluate(val_dataset, verbose=0)
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

print("Saving final model...")
final_model.save(os.path.join(output_folder, "ResNet50_model_2d.h5"))
print("Model saved successfully.")


Mounting Google Drive...
Mounted at /content/drive
Loading preprocessed data...
Data loading complete. Loaded 6400 images.
Performing train-validation split...
Defining data augmentation generator...
Setting up distributed training strategy using MirroredStrategy...
Generation 0
Evaluating model with learning rate: 0.000642373999870473
Creating ResNet model with learning rate: 0.000642373999870473
Evaluating model with learning rate: 0.000475686114173182
Creating ResNet model with learning rate: 0.000475686114173182
Evaluating model with learning rate: 0.00013650886869610363
Creating ResNet model with learning rate: 0.00013650886869610363
Evaluating model with learning rate: 0.0009377324655767442
Creating ResNet model with learning rate: 0.0009377324655767442
Evaluating model with learning rate: 0.00033697795686054715
Creating ResNet model with learning rate: 0.00033697795686054715
Evaluating model with learning rate: 0.0002601131279809896
Creating ResNet model with learning rate: 0.00

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved successfully.


PART 2.4 full code (VGG16)

In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from google.colab import drive
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import gc
from deap import base, creator, tools

# Mount Google Drive
print("Mounting Google Drive...")
drive.mount('/content/drive')

# Define data dimensions
width, height = 224, 224

# Data paths
preprocessed_folder = "/content/drive/MyDrive/AD_Dataset_Preprocessed"
output_folder = "/content/drive/MyDrive/Models"
num_epochs = 20
batch_size = 32

# Mapping class names to labels
class_name_to_label = {'VeryMildDemented': 0, 'MildDemented': 1, 'ModerateDemented': 2, 'NonDemented': 3}

# Function to load preprocessed data
def load_preprocessed_data(preprocessed_folder):
    data, labels = [], []
    for class_folder in os.listdir(preprocessed_folder):
        class_label = class_name_to_label.get(class_folder, -1)
        if class_label != -1:
            class_folder_path = os.path.join(preprocessed_folder, class_folder)
            for file_name in os.listdir(class_folder_path):
                if file_name.endswith(".png"):
                    image_path = os.path.join(class_folder_path, file_name)
                    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
                    image = cv2.resize(image, (width, height))
                    # Convert grayscale to RGB
                    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
                    data.append(image)
                    labels.append(class_label)

    data = np.array(data)
    labels = np.array(labels)
    print(f"Data loading complete. Loaded {len(data)} images.")
    return data, labels

print("Loading preprocessed data...")
data, labels = load_preprocessed_data(preprocessed_folder)

print("Performing train-validation split...")
train_data, val_data, train_labels, val_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

print("Defining data augmentation generator...")
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

num_classes = len(np.unique(train_labels))

def create_model(learning_rate):
    print(f"Creating VGG16 model with learning rate: {learning_rate}")
    base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(width, height, 3))
    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

print("Setting up distributed training strategy...")
strategy = tf.distribute.MirroredStrategy()

def preprocess_for_tf_datagen(image, label):
    """Preprocesses an image for TensorFlow data augmentation.

    Args:
        image: A tensor representing an image.
        label: A tensor representing the image label.

    Returns:
        A tuple of (image, label) where the image is preprocessed.
    """
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    image = tf.image.resize(image, [width, height])
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    image = tf.image.random_brightness(image, 0.2)
    return image, label

with strategy.scope():
    train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels)).map(
        preprocess_for_tf_datagen
    ).batch(batch_size).prefetch(tf.data.AUTOTUNE)

    val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_labels)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

def lr_schedule(epoch):
    return 0.001 * (0.1 ** int(epoch / 10))

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

def evaluate_model(individual):
    learning_rate = individual[0]
    print(f"Evaluating model with learning rate: {learning_rate}")
    model = create_model(learning_rate)
    model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.LearningRateScheduler(lr_schedule), early_stopping], verbose=0)
    _, val_accuracy = model.evaluate(val_dataset, verbose=0)
    del model
    gc.collect()  # Manually trigger garbage collection to clear memory
    return -val_accuracy,

del data
del labels
del train_data
del val_data

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register("attr_float", np.random.uniform, 1e-5, 1e-3)  # Learning rate range
toolbox.register("individual", tools.initCycle, creator.Individual, (toolbox.attr_float,), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

population = toolbox.population(n=10)

hall_of_fame = tools.HallOfFame(1)

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("max", np.max)
stats.register("avg", np.mean)

NGEN = 1  # Number of generations
CXPB = 0.7  # Crossover probability
MUTPB = 0.2  # Mutation probability

logbook = tools.Logbook()
logbook.header = ["gen", "evals", "max", "avg"]

for gen in range(NGEN):
    print(f"Generation {gen}")
    fitnesses = map(evaluate_model, population)
    for ind, fit in zip(population, fitnesses):
        ind.fitness.values = fit

    hall_of_fame.update(population)

    offspring = list(map(toolbox.clone, population))

    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if np.random.rand() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if np.random.rand() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    fitnesses = map(evaluate_model, offspring)
    for ind, fit in zip(offspring, fitnesses):
        ind.fitness.values = fit

    population[:] = offspring

    record = stats.compile(population)
    logbook.record(gen=gen, evals=len(population), **record)
    print(logbook.stream)

best_ind = hall_of_fame[0]
best_fitness = best_ind.fitness.values[0]
print("Best Individual:")
print("Learning Rate =", best_ind[0])
print("Best Fitness =", best_fitness)

best_learning_rate = best_ind[0]
with strategy.scope():
    final_model = create_model(best_learning_rate)

print("Training final model...")
final_model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.LearningRateScheduler(lr_schedule), early_stopping])

print("Evaluating final model...")
val_loss, val_accuracy = final_model.evaluate(val_dataset, verbose=0)
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

print("Saving final model...")
final_model.save(os.path.join(output_folder, "VGG16_model_3d.h5"))
print("Model saved successfully.")


Mounting Google Drive...
Mounted at /content/drive
Loading preprocessed data...
Data loading complete. Loaded 6400 images.
Performing train-validation split...
Defining data augmentation generator...
Setting up distributed training strategy...
Generation 0
Evaluating model with learning rate: 0.0008385895678256894
Creating VGG16 model with learning rate: 0.0008385895678256894
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Evaluating model with learning rate: 0.0009633527934635357
Creating VGG16 model with learning rate: 0.0009633527934635357
Evaluating model with learning rate: 0.0006125543937651937
Creating VGG16 model with learning rate: 0.0006125543937651937
Evaluating model with learning rate: 0.0007919640772930794
Creating VGG16 model with learning rate: 0.0007919640772930794
Evaluating model with learning rate: 0.00010590150852849701
Creating VGG16 model with learning rate: 0.00010590150852849701
Evaluating model with learning rate: 0.0008512854727659575
Creating VGG16 model 

Validation Loss: 1.3433
Validation Accuracy: 52.11%
Saving final model...
Model saved successfully.
